# Neighborhood Effects of Trade in Landlocked Countries

The data set was part of my undergraduate senior project. The dataset contains data for 30 landlocked coutries over five years from 2014 to 2018. The data was obtained from the World Bank's World Development Indicators database and its World Inegrated Trade Services database. The goal of the project was to look at the effects of trade with neighboring countries on the economic level of a landlocked country.

In [1]:
import pandas as pd

# User-defined Methods

These methods were created in order to handle repitive tasks and also to make the code more readable.

In [2]:
# change column into categorical
def set_as_category(df, col):
    df[col] = df[col].astype("category")
    return df

# remap values of income level
def refactor_income(df):
    mapping={'H': 1, 'UM': 1, 'LM': 0, 'L': 0}
    df['income'] = df['income'].map(mapping)
    return df

# increase number of rows; change year from columns to rows
def melt(df, col_name):
    return pd.melt(df, id_vars = 'Country Code', value_vars = range(2014, 2019), var_name = 'year', value_name = col_name)

# make sure that column header for years are the same
def rename_fy_year(df):
    for col in df.columns:
        if col[slice(4)].isdigit():
            df.rename(columns = {col:int(col[slice(4)])}, inplace = True)

    return df

# function that changes non-numeric values to NAN
def non_num_na(df):
    for i in df.columns[2:]:
        df[i] = pd.to_numeric(df[i], errors = 'coerce')
    return df

## Data exploration

An array was created with the names of data frames and a for loop was used to read all the files to the data frames. 

In [3]:
tables = ['gdp_pc', 'inflation', 'trade_pct', 'unemployment', 'income', 'total_export', 'export_neighbors', 'import_neighbors']

# loop to create dataframes from csv files
for table in tables:
    locals()[table] = pd.read_csv("datafiles/" + table + ".csv")\
        .pipe(rename_fy_year)\
        .pipe(melt, col_name = table)\
        .sort_values(by = ['Country Code', 'year'])

### Neighborhood trade factors

`trade_ratio` and `trade_share` were created as indices to quantify the trade of a landlocked countries with their neighboring countries.

`trade_ratio` is calculated as $\frac{export_neighbors}{import_neighbors}$ and is the ratio of export and import with neighboring countries. A value of greater than $1$ indicates that this a landlocked country exports more to it neighbors than it imports from them and a value less than $1$ indicates the opposite.

`trade_share` is calculates as $\frac{export_neighbors}{total_export}$ and is the ratio of export to neighboring countries and the overall export by a landlocked country. A higher value indicates that the share of trade with neighboring countries is high for the landlocked country.

In [4]:
trade = pd.merge(total_export, export_neighbors)
trade = pd.merge(trade, import_neighbors)

# calculate trade_ratio and trade_share
trade['trade_ratio'] = trade.export_neighbors / trade.import_neighbors
trade['trade_share'] = trade.export_neighbors / trade.total_export
trade = trade.drop(['total_export', 'export_neighbors', 'import_neighbors'], axis = 1)

# Joining Tables

The separate tables are joined using the `dataFrame.merge()` method. 

In [5]:
# merge into a single dataframe
landlocked = pd.merge(gdp_pc, trade)
landlocked = pd.merge(landlocked, inflation)
landlocked = pd.merge(landlocked, trade_pct)
landlocked = pd.merge(landlocked, unemployment)
landlocked = pd.merge(landlocked, income)\
    .pipe(set_as_category, col = 'Country Code')\
    .pipe(set_as_category, col = 'year')\
    .pipe(refactor_income)\
    .pipe(set_as_category, col = 'income')\
    .pipe(non_num_na)\
    .fillna(method="ffill")

In [6]:
# write to file
landlocked.to_csv("datafiles/landlocked.csv")